# White light curve

This is breakdown of the SOSSISSE white light curve functionality. 

It is here an an illustration only.

Please use the sosssisse.white_light_curve function for processing data.

## Setup of SOSSISSE

In [ ]:
import sossisse

# either define a yaml parameter file and/or define parameters here
param_file = 'my_parameters.yaml'

pkwargs = dict()
# For example to set or override the data directory (SOSSIOPATH) uncomment this line
# pkwargs['SOSSIOPATH'] = '/scratch3/jwst/sossisse-data/demo/'

# get parameters
inst = sossisse.get_parameters(param_file=param_file, **pkwargs)

## White light curve functions

The following is a copy and paste of the `sossisse.general.general.white_light_curve` function.

In [ ]:
# imports
import os

from sossisse.core import io
from sossisse.core import misc
from sossisse.general import plots

### Pre-amble - setting up variables etc

In [ ]:
# set the function name
func_name = 'white_light_curve'
# print the splash
misc.sossart()
# get parameters from instrumental parameters
objname = inst.params['OBJECTNAME']
# print the white light curve splash
print(misc.art('White light curve ' + objname, 'blue', 'CYAN'))
# -------------------------------------------------------------------------
# load temporary filenames (should be run before science starts)
inst.define_filenames()
wlc_ltbl_file = inst.get_variable('WLC_LTBL_FILE', func_name)
# return if we have the soss_stablity file
if os.path.exists(wlc_ltbl_file):
    msg = 'File {0} exists we skip white light curve step'
    misc.printc(msg.format(wlc_ltbl_file), 'info')

### Load the data

In [ ]:
# load the image, error and data quality
cube, err, dq = inst.load_data_with_dq()

### Remove the background

In [ ]:
# remove the background
cube, err = inst.remove_background(cube, err, dq)

### Patch isolated bad pixels

In [ ]:
# for each slice of the cube, isolated bad pixels are interpolated with the
# value of their 4 neighbours.
cube = inst.patch_isolated_bads(cube)

### Get the trace map

In [ ]:
# get the trace map
tracemap = inst.get_trace_map()

### Clean 1/f noise

In [ ]:
# if you want to subtract a higher order polynomial to the 1/f noise, change
# the value of fit_order
out_c1f = inst.clean_1f(cube, err, tracemap)
cube, med, med_diff, transit_invsout, pcas = out_c1f

### Recenter the trace position

In [ ]:
# recenter the trace position
tracemap = inst.recenter_trace_position(tracemap, med)

### Rotation/Shift trace correction

In [ ]:
# get the gradients
dx, dy, rotxy, ddy, med_clean = inst.get_gradients(med)
# set up the mask for trace position
mask_out = inst.get_mask_trace_pos(med, tracemap)
mask_trace_pos, x_order0, y_order0, x_trace_pos, y_trace_pos = mask_out

### Linear reconstruction of amplitudes

In [ ]:
# setup the linear reconstruction vector based on the input parameters
lvector = inst.setup_linear_reconstruction(med, dx, dy, rotxy, ddy,
                                           pcas, med_diff)
# find the best linear combination of scale/dx/dy/rotation from lvector
# amps is a vector with the amplitude of all 4 fitted terms
# amps[0] -> amplitude of trace
# amps[1] -> dx normalized on reference trace
# amps[2] -> dy normalized on reference trace
# amps[3] -> rotation (in radians) normalized on reference trace
# amps[4] -> 2nd derivative in y [if option activated]
# -------------------------------------------------------------------------
amp_out = inst.apply_amp_recon(cube, err, med, mask_trace_pos,
                               lvector, x_trace_pos, y_trace_pos,
                               x_order0, y_order0)
# get outputs of apply_amp_recon
ltable, lrecon, valid_cube = amp_out

### Normalize the trace

In [ ]:
# normalize the trace but a normalization factor
ltable = inst.normalize_sum_trace(ltable)

### Per pixel baseline correction

In [ ]:
# per pixel baseline
if inst.params['PER_PIXEL_BASELINE_CORRECTION']:
    misc.printc('Performing per-pixel baseline subtraction', 'info')
    cube = inst.per_pixel_baseline(cube, valid_cube)

### RMS of baseline

In [ ]:
# print the rms baseline for all methods
for method in inst.get_rms_baseline():
    # calculate the rms for this method
    rms_method = inst.get_rms_baseline(ltable['amplitude'], method=method)
    # print this
    msg = '{0}, rms = {1:.1f}ppm'.format(method, rms_method * 1e6)
    misc.printc(msg, 'number')

### Get the effective wavelength

In [ ]:
# get the effective wavelength
photo_weighted_mean, energy_weighted_mean = inst.get_effective_wavelength()

### Write files + plot + summary html

In [ ]:
# =========================================================================
# write files
# =========================================================================
inst.save_wlc_results(cube, err, lrecon, ltable)
# =========================================================================
# Plots and Summary HTML
# =========================================================================
# plot the stability plot
plots.plot_stability(inst, ltable)
# -------------------------------------------------------------------------
# plot the transit plot
plots.plot_transit(inst, ltable)
# -------------------------------------------------------------------------
# write the yaml file to html
io.summary_html(inst.params)